In [1]:
import pyspark

from pyspark.sql import SparkSession # importting spark context( basic builder element for Dataframe API)

spark = SparkSession.builder.appName('basics').getOrCreate() 

In [2]:
import numpy as np
import pandas as pd 

from pyspark.sql.types import *  # Necessary for creating schemas
from pyspark.sql.functions import * # Importing PySpark functions

from pyspark.sql.window import Window # for partitionby

In [3]:
df = spark.sql('select * from ginni_csv')

#spark.read.csv('/FileStore/tables/fakefriends.csv')

In [4]:
df.show(2)

+----+-----+------+
Name|sales|profit|
+----+-----+------+
 A1| 6805| 49|
 A2| 6110| 28|
+----+-----+------+
only showing top 2 rows

In [5]:
df.describe().show()

+-------+----+------------------+------------------+
summary|Name| sales| profit|
+-------+----+------------------+------------------+
 count| 25| 25| 25|
 mean|null| 5368.4| 51.68|
 stddev|null|2159.6297329249137|24.489317398953094|
 min| A1| 1431| 100|
 max| A9| 9951| 98|
+-------+----+------------------+------------------+

In [6]:
#### Creating lead/lag 

# df.select(lag('sales',1).over(Window.partitionBy('Name').orderBy('Name'))).show()
df.select(lag('sales',1).over(Window.orderBy('Name')).alias('lead_lag')).show(2)

+--------+
lead_lag|
+--------+
 null|
 6805|
+--------+
only showing top 2 rows

In [7]:
#### Ranking within a group - 2nd highest element

df1 = df.unionAll(df)

windowval = (Window.partitionBy('Name').orderBy(desc('sales')))

counter = df1.select('Name',row_number().over(windowval).alias('row_count'))
counter.filter(col('row_count') == 2).show(2)

+----+---------+
Name|row_count|
+----+---------+
 A23| 2|
 A9| 2|
+----+---------+
only showing top 2 rows

In [8]:
#### Adding index to a spark DF - This is the way, although on databricks its showing import error!

# from pyspark.sql.functions import monotonicallyIncreasingId

# df.select(monotonicallyIncreasingId().alias("rowId"),"*")

In [9]:
# adding row num :: but first by ordering
df.select(row_number().over(Window.orderBy('sales')).alias('row_num'),'sales').show(5)

+-------+-----+
row_num|sales|
+-------+-----+
 1| 1431|
 2| 2222|
 3| 3248|
 4| 3317|
 5| 3418|
+-------+-----+
only showing top 5 rows

In [10]:
#### Transposing || Apply function

# Transposing a DF in spark in a pain!
# Implementing apply like function(that iterates through rows/columns) is a pain! ::: This requires conversion to RDDs first

In [11]:
# storing a pyspark DF in a np array : if we want to calc rowwise maximum or Transpose etc

np.array(df.select('*').collect())[:10]

Out[ 116 ]: array([['A1', '6805', '49'],
 ['A2', '6110', '28'],
 ['A3', '7888', '88'],
 ['A4', '4773', '28'],
 ['A5', '2222', '100'],
 ['A6', '5489', '51'],
 ['A7', '9951', '91'],
 ['A8', '6673', '62'],
 ['A9', '8987', '25'],
 ['A10', '7057', '48']], 
 dtype='<U4')

In [12]:
#### Intersection between 2 Dataframes

x = df.select('name').intersect(df.select('name')) 
x.show(2)

+----+
name|
+----+
 A23|
 A9|
+----+
only showing top 2 rows

In [13]:
#### Diference in values in 1 column WRT other column

x = df[['Name']].limit(20)

df.join(x,df['name'] == x['name'],'left_anti').show()

+----+-----+------+
Name|sales|profit|
+----+-----+------+
 A21| 4936| 21|
 A22| 3450| 45|
 A23| 4665| 58|
 A24| 3248| 34|
 A25| 3553| 36|
+----+-----+------+

In [14]:
#### Rolling mean
https://stackoverflow.com/questions/45806194/pyspark-rolling-average-using-timeseries-data

In [15]:
### Label encoding

from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCol="Name", outputCol="index_name")
indexed = indexer.fit(df).transform(df)
indexed.show(2)

+----+-----+------+----------+
Name|sales|profit|index_name|
+----+-----+------+----------+
 A1| 6805| 49| 16.0|
 A2| 6110| 28| 14.0|
+----+-----+------+----------+
only showing top 2 rows

In [16]:
#### obtaining duplicate rows/sunsets

# +---+-----+------+----+------------+------------+
# | ID|  ID2|Number|Name|Opening_Hour|Closing_Hour|
# +---+-----+------+----+------------+------------+
# |ALT|  QWA|     6|null|    08:59:00|    23:30:00|
# |ALT|AUTRE|     2|null|    08:58:00|    23:29:00|
# |TDR|  QWA|     3|null|    08:57:00|    23:28:00|
# |ALT| TEST|     4|null|    08:56:00|    23:27:00|
# |ALT|  QWA|     6|null|    08:55:00|    23:26:00|
# |ALT|  QWA|     2|null|    08:54:00|    23:25:00|
# |ALT|  QWA|     2|null|    08:53:00|    23:24:00|
# +---+-----+------+----+------------+------------+



# https://stackoverflow.com/questions/49559994/keep-only-duplicates-from-a-dataframe-regarding-some-field
  
# w = Window.partitionBy('ID', 'ID2', 'Number')
# df.select('*', f.count('ID').over(w).alias('dupeCount'))\
#     .where('dupeCount > 1')\
#     .drop('dupeCount')\
#     .show()